**2020 Election and Vaccination Hesitancy Data**

Annie Bishai

In [2]:
import pandas as pd
import numpy as np

# 2020 election data

In [3]:
df = pd.read_csv('./source_data/SRC_president_county_candidate.csv')
#source = https://www.kaggle.com/ajaypalsinghlo/us-election-2020-president-county-candidate/data?select=president_county_candidate.csv

In [4]:
df.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [5]:
df.shape

(32177, 6)

In [6]:
state_dict = {
  'Alabama': 'AL',
    'Alaska': 'AK', 
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
df['state'] = df['state'].map(state_dict)

In [7]:
df['county'] = df['county'] + '_' + df['state']
df.head()

,state,county,candidate,party,total_votes,won
0,DE,Kent County_DE,Joe Biden,DEM,44552,True
1,DE,Kent County_DE,Donald Trump,REP,41009,False
2,DE,Kent County_DE,Jo Jorgensen,LIB,1044,False
3,DE,Kent County_DE,Howie Hawkins,GRN,420,False
4,DE,New Castle County_DE,Joe Biden,DEM,195034,True


In [8]:
len(df['county'].unique())

4633

In [9]:
winners = df[df['won'] == True].copy()
winners.set_index('county', inplace=True)
winners.rename(columns={'candidate':'candidate_won', 'total_votes':'winner_votes'}, inplace=True)
winners.drop(columns='won', inplace=True)

In [10]:
winners.head()

,state,candidate_won,party,winner_votes
county,,,,
Kent County_DE,DE,Joe Biden,DEM,44552
New Castle County_DE,DE,Joe Biden,DEM,195034
Sussex County_DE,DE,Donald Trump,REP,71230
District of Columbia_DC,DC,Joe Biden,DEM,39041
Ward 2_DC,DC,Joe Biden,DEM,29078


In [11]:
winners.shape

(4633, 4)

In [12]:
loser_votes = df[df['won'] == False].copy().groupby('county').sum()
loser_votes.drop(columns='won', inplace=True)
loser_votes.rename(columns={'total_votes': 'nonwinner_votes'}, inplace=True)
loser_votes.head()

,nonwinner_votes
county,
Abbeville County_SC,4218
Abbot_ME,129
Abington_MA,4451
Acadia Parish_LA,5829
Accomack County_VA,7790


In [13]:
votes = pd.merge(winners, loser_votes, left_on = 'county', right_on = 'county')

In [14]:
votes.shape

(4633, 5)

In [15]:
votes['total_votes'] = votes['winner_votes'] + votes['nonwinner_votes']
votes.head()

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes
county,,,,,,
Kent County_DE,DE,Joe Biden,DEM,44552,42473,87025
New Castle County_DE,DE,Joe Biden,DEM,195034,92599,287633
Sussex County_DE,DE,Donald Trump,REP,71230,58122,129352
District of Columbia_DC,DC,Joe Biden,DEM,39041,2640,41681
Ward 2_DC,DC,Joe Biden,DEM,29078,3803,32881


In [16]:
trump = votes[votes['candidate_won'] == 'Donald Trump'].copy()
trump['pct_trump'] = round(trump['winner_votes'] / trump['total_votes'], 3)
trump['pct_biden'] = round(trump['nonwinner_votes'] / trump['total_votes'], 3)
trump.shape

(3219, 8)

In [17]:
biden = votes[votes['candidate_won'] == 'Joe Biden'].copy()
biden['pct_trump'] = round(biden['nonwinner_votes'] / biden['total_votes'], 3)
biden['pct_biden'] = round(biden['winner_votes'] / biden['total_votes'], 3)
biden.shape

(1410, 8)

In [18]:
df_comb = pd.concat(objs=[biden, trump])
df_comb.head()

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county,,,,,,,,
Kent County_DE,DE,Joe Biden,DEM,44552,42473,87025,0.488,0.512
New Castle County_DE,DE,Joe Biden,DEM,195034,92599,287633,0.322,0.678
District of Columbia_DC,DC,Joe Biden,DEM,39041,2640,41681,0.063,0.937
Ward 2_DC,DC,Joe Biden,DEM,29078,3803,32881,0.116,0.884
Ward 3_DC,DC,Joe Biden,DEM,39397,4834,44231,0.109,0.891


In [19]:
df_comb.shape

(4629, 8)

In [20]:
# ID length discrepancy
other = votes[(votes['candidate_won'] != 'Joe Biden') & (votes['candidate_won'] != 'Donald Trump')].copy()
other

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes
county,,,,,,
Hersey_ME,ME,Jo Jorgensen,LIB,33,9,42
ED 26_AK,AK,Write-ins,WRI,10181,10111,20292
ED 27_AK,AK,Write-ins,WRI,9531,9506,19037
ED 28_AK,AK,Write-ins,WRI,12831,12749,25580


In [21]:
# grab data from counties that voted for 3rd party or write-in
third_party = df[df['county'].map(lambda x: x in ['Hersey_ME', 'ED 26_AK', 'ED 27_AK', 'ED 28_AK'])]
third_party[third_party['candidate'].map(lambda x: x in ['Donald Trump', 'Joe Biden'])]

,state,county,candidate,party,total_votes,won
5837,ME,Hersey_ME,Joe Biden,DEM,8,False
5839,ME,Hersey_ME,Donald Trump,REP,0,False
28363,AK,ED 26_AK,Donald Trump,REP,5243,False
28364,AK,ED 26_AK,Joe Biden,DEM,4558,False
28371,AK,ED 27_AK,Joe Biden,DEM,4844,False
28372,AK,ED 27_AK,Donald Trump,REP,4324,False
28379,AK,ED 28_AK,Joe Biden,DEM,6264,False
28380,AK,ED 28_AK,Donald Trump,REP,6162,False


In [22]:
other[['pct_biden', 'pct_trump']] = 0
other.loc['Hersey_ME', ['pct_biden', 'pct_trump']] = (round(8/42, 3), 0)
other.loc['ED 26_AK', ['pct_biden', 'pct_trump']] = (round(4558/20292, 3), round(5243/20292, 3))
other.loc['ED 27_AK', ['pct_biden', 'pct_trump']] = (round(4844/19037, 3), round(4324/19037, 3))
other.loc['ED 28_AK', ['pct_biden', 'pct_trump']] = (round(6264/25580, 3), round(6162/25580, 3))

In [23]:
other

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_biden,pct_trump
county,,,,,,,,
Hersey_ME,ME,Jo Jorgensen,LIB,33,9,42,0.190,0.000
ED 26_AK,AK,Write-ins,WRI,10181,10111,20292,0.225,0.258
ED 27_AK,AK,Write-ins,WRI,9531,9506,19037,0.254,0.227
ED 28_AK,AK,Write-ins,WRI,12831,12749,25580,0.245,0.241


In [24]:
df_final = pd.concat(objs=[df_comb, other])
df_final.shape

(4633, 8)

In [25]:
df_final.to_csv('./clean_data/2020_election_by_county.csv')

In [26]:
## for DC combination below:
trump_dc = df[(df['state'] == 'DC') & (df['candidate'] == 'Donald Trump')][['total_votes']].sum()

# Vaccine hesitancy data

In [27]:
vh = pd.read_csv('./source_data/SRC_cleaned_vaccine_03.08.2022.csv')
# https://www.healthdata.org/acting-data/covid-19-vaccine-hesitancy-us-county-and-zip-code
vh.head()

,week,start_date,end_date,zip_code,vaccine_measure_id,final_zip_pred,state_name,county_name,vaccine_measure_name,definition
0,60,2022-02-18,2022-02-24,10001,1,0.025570,New York,New York County,high_vaccine_potential,yes probably will and no probably wont respond...
1,60,2022-02-18,2022-02-24,10002,1,0.024326,New York,New York County,high_vaccine_potential,yes probably will and no probably wont respond...
2,60,2022-02-18,2022-02-24,10003,1,0.012319,New York,New York County,high_vaccine_potential,yes probably will and no probably wont respond...
3,60,2022-02-18,2022-02-24,10004,1,0.020720,New York,New York County,high_vaccine_potential,yes probably will and no probably wont respond...
4,60,2022-02-18,2022-02-24,10005,1,0.018961,New York,New York County,high_vaccine_potential,yes probably will and no probably wont respond...


In [28]:
vh.shape

(88348, 10)

In [29]:
vh['state_name'] = vh['state_name'].map(state_dict)

In [30]:
vh['county_name'] = vh['county_name'] + '_' + vh['state_name']

In [31]:
len(vh['county_name'].unique())

3142

In [32]:
vh.dtypes

week                      int64
start_date               object
end_date                 object
zip_code                  int64
vaccine_measure_id        int64
final_zip_pred          float64
state_name               object
county_name              object
vaccine_measure_name     object
definition               object
dtype: object

In [33]:
from datetime import datetime

# Function to convert
def get_datetime(date):
    return datetime.strptime(date, '%Y-%M-%d').date()


In [34]:
vh['definition'].value_counts()

yes probably will and no probably wont respondents                         44174
yes probably will, no probably wont, and no definitely wont respondents    44174
Name: definition, dtype: int64

In [35]:
vh['vaccine_measure_name'].value_counts()

high_vaccine_potential    44174
all_vaccine_potential     44174
Name: vaccine_measure_name, dtype: int64

### Divide into separate dataframes for separate survey measures

In [36]:
vh_potential = vh[vh['vaccine_measure_name'] == 'high_vaccine_potential'].copy()

In [37]:
vh_no = vh[vh['vaccine_measure_name'] == 'all_vaccine_potential'].copy()

In [38]:
len(vh_no['county_name'].unique())

3142

**Create features based on county aggregates of "all vaccine potential" survey info**

In [39]:
low_conf_measures = vh_no.groupby('county_name')[['final_zip_pred']].describe()
low_conf_measures

final_zip_pred                                          \
                             count      mean       std       min       25%   
county_name                                                                  
Abbeville County_SC           13.0  0.292742  0.078290  0.172745  0.206141   
Acadia Parish_LA              15.0  0.289879  0.048135  0.219274  0.264155   
Accomack County_VA            33.0  0.213604  0.024754  0.146208  0.199243   
Ada County_ID                 17.0  0.165158  0.067930  0.088074  0.132037   
Adair County_IA               13.0  0.289593  0.081429  0.182733  0.221080   
...                            ...       ...       ...       ...       ...   
Yuma County_AZ                10.0  0.140358  0.028423  0.087786  0.124811   
Yuma County_CO                10.0  0.232075  0.071833  0.166699  0.189485   
Zapata County_TX               5.0  0.131007  0.016373  0.103797  0.127515   
Zavala County_TX               3.0  0.204671  0.014042  0.192347  0.197027   
Ziebach County_SD              7.0  0.225771  0.058247  0.155204  0.185255   

                                                   
                          50%       75%       max  
county_name                                        
Abbeville County_SC  0.314208  0.347584  0.405147  
Acadia Parish_LA     0.278279  0.304953  0.428974  
Accomack County_VA   0.212687  0.225692  0.289476  
Ada County_ID        0.151415  0.185319  0.378846  
Adair County_IA      0.300198  0.326022  0.426109  
...                       ...       ...       ...  
Yuma County_AZ       0.142352  0.159289  0.175460  
Yuma County_CO       0.206782  0.255156  0.398401  
Zapata County_TX     0.140173  0.140391  0.143159  
Zavala County_TX     0.201708  0.210833  0.219959  
Ziebach County_SD    0.207884  0.264219  0.318362  

[3142 rows x 8 columns]

In [40]:
low_conf_measures.columns = low_conf_measures.columns.droplevel()
    # https://stackoverflow.com/questions/22233488/pandas-drop-a-level-from-a-multi-level-column-index

In [41]:
low_conf_measures.drop(columns=['min', 'std', '25%', '50%', '75%', 'max'], inplace=True)
low_conf_measures.rename(columns={'count': 'n_zipcodes', 'mean': 'pct_hesitant_02-24-22'},inplace=True)

**Repeat for "high vaccine potential" survey info**

In [42]:
low_mid_conf_measures = vh_potential.groupby('county_name')[['final_zip_pred']].describe()
#low_mid_conf_measures

In [43]:
low_mid_conf_measures.columns = low_mid_conf_measures.columns.droplevel()

In [44]:
low_mid_conf_measures.drop(columns=['count', 'std', 'min', '25%', '50%', '75%', 'max'], inplace=True)
low_mid_conf_measures.rename(columns={'mean': 'pct_somewhat_hesitant_02-24-22'},inplace=True)

In [45]:
low_mid_conf_measures

,pct_somewhat_hesitant_02-24-22
county_name,
Abbeville County_SC,0.108014
Acadia Parish_LA,0.082989
Accomack County_VA,0.087405
Ada County_ID,0.037143
Adair County_IA,0.058975
...,...
Yuma County_AZ,0.037530
Yuma County_CO,0.062108
Zapata County_TX,0.043724


**Merge and create high-hesitancy feature**

In [46]:
hesitancy = pd.merge(low_conf_measures, low_mid_conf_measures, left_on='county_name', right_on='county_name')
hesitancy.head()

,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22
county_name,,,
Abbeville County_SC,13.0,0.292742,0.108014
Acadia Parish_LA,15.0,0.289879,0.082989
Accomack County_VA,33.0,0.213604,0.087405
Ada County_ID,17.0,0.165158,0.037143
Adair County_IA,13.0,0.289593,0.058975


In [47]:
hesitancy['pct_highly_hesitant_02-24-22'] = hesitancy['pct_hesitant_02-24-22'] - hesitancy['pct_somewhat_hesitant_02-24-22']

In [48]:
hesitancy.head()

,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22,pct_highly_hesitant_02-24-22
county_name,,,,
Abbeville County_SC,13.0,0.292742,0.108014,0.184729
Acadia Parish_LA,15.0,0.289879,0.082989,0.206890
Accomack County_VA,33.0,0.213604,0.087405,0.126199
Ada County_ID,17.0,0.165158,0.037143,0.128015
Adair County_IA,13.0,0.289593,0.058975,0.230618


In [49]:
hesitancy.shape

(3142, 4)

In [50]:
hesitancy.to_csv('./clean_data/vaccine_hesitancy.csv')

# Clean for merging

In [56]:
vote = df_final.copy()
vax = hesitancy.copy()

In [60]:
vax.head()

,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22,pct_highly_hesitant_02-24-22
county_name,,,,
Abbeville County_SC,13.0,0.292742,0.108014,0.184729
Acadia Parish_LA,15.0,0.289879,0.082989,0.206890
Accomack County_VA,33.0,0.213604,0.087405,0.126199
Ada County_ID,17.0,0.165158,0.037143,0.128015
Adair County_IA,13.0,0.289593,0.058975,0.230618


### county_state formatting

In [59]:
vote.index = vote.index.map(lambda x : x.replace(' County',''))


In [61]:
vax.index = vax.index.map(lambda x : x.replace(' County',''))


In [62]:
# create lowercase county_state, set as index

vax['county_state'] = vax.index.str.lower()
vote['county_state'] = vote.index.str.lower()

vax.set_index('county_state', inplace=True)
vote.set_index('county_state', inplace=True)

In [64]:
# drop unnecessary columns

#vax.drop(columns='county_name', inplace=True)
vax['pct_hesitant_02-24-22'] = vax['pct_hesitant_02-24-22'].map(lambda x: round(x, 3))
vax['pct_somewhat_hesitant_02-24-22'] = vax['pct_somewhat_hesitant_02-24-22'].map(lambda x: round(x, 3))
vax['pct_highly_hesitant_02-24-22'] = vax['pct_highly_hesitant_02-24-22'].map(lambda x: round(x, 3))
vax.head()

,n_zipcodes,pct_hesitant_02-24-22,pct_somewhat_hesitant_02-24-22,pct_highly_hesitant_02-24-22
county_state,,,,
abbeville_sc,13.0,0.293,0.108,0.185
acadia parish_la,15.0,0.290,0.083,0.207
accomack_va,33.0,0.214,0.087,0.126
ada_id,17.0,0.165,0.037,0.128
adair_ia,13.0,0.290,0.059,0.231


### Remove DC wards from voting df

In [65]:
[x for x in vax.index if x.endswith('DC')]

[]

In [67]:
vote.head()

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,,
kent_de,DE,Joe Biden,DEM,44552,42473,87025,0.488,0.512
new castle_de,DE,Joe Biden,DEM,195034,92599,287633,0.322,0.678
district of columbia_dc,DC,Joe Biden,DEM,39041,2640,41681,0.063,0.937
ward 2_dc,DC,Joe Biden,DEM,29078,3803,32881,0.116,0.884
ward 3_dc,DC,Joe Biden,DEM,39397,4834,44231,0.109,0.891


In [68]:
dc = vote.loc[['district of columbia_dc',
 'ward 2_dc',
 'ward 3_dc',
 'ward 4_dc',
 'ward 5_dc',
 'ward 6_dc',
 'ward 7_dc',
 'ward 8_dc']].copy()
dc

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,,
district of columbia_dc,DC,Joe Biden,DEM,39041,2640,41681,0.063,0.937
ward 2_dc,DC,Joe Biden,DEM,29078,3803,32881,0.116,0.884
ward 3_dc,DC,Joe Biden,DEM,39397,4834,44231,0.109,0.891
ward 4_dc,DC,Joe Biden,DEM,42489,2807,45296,0.062,0.938
ward 5_dc,DC,Joe Biden,DEM,43320,2787,46107,0.060,0.940
ward 6_dc,DC,Joe Biden,DEM,56719,6199,62918,0.099,0.901
ward 7_dc,DC,Joe Biden,DEM,36382,2002,38384,0.052,0.948
ward 8_dc,DC,Joe Biden,DEM,30897,1961,32858,0.060,0.940


In [69]:
dc_winner_votes = dc['winner_votes'].sum()
dc_nonwinner_votes = dc['nonwinner_votes'].sum()
dc_total_votes = dc['total_votes'].sum()
dc_pct_biden = dc['winner_votes'].sum() / dc['total_votes'].sum()
dc_pct_trump = trump_dc / dc['total_votes'].sum()

In [70]:
vote.drop([
 'ward 2_dc',
 'ward 3_dc',
 'ward 4_dc',
 'ward 5_dc',
 'ward 6_dc',
 'ward 7_dc',
 'ward 8_dc'], inplace=True)

In [71]:
vote.loc['district of columbia'] = ['DC', 'Joe Biden', 'DEM', dc_winner_votes, dc_nonwinner_votes, dc_total_votes, dc_pct_trump, dc_pct_biden]

In [72]:
vote.loc['district of columbia']

state                                                  DC
candidate_won                                   Joe Biden
party                                                 DEM
winner_votes                                       317323
nonwinner_votes                                     27033
total_votes                                        344356
pct_trump          total_votes    0.053973
dtype: float64
pct_biden                                        0.921497
Name: district of columbia, dtype: object

In [73]:
vote.to_csv('./clean_data/2020_election_by_county.csv')
vax.to_csv('./clean_data/vaccine_hesitancy.csv')

## Handle mismatch of rows in vote and vax dfs

In [74]:
vote = pd.read_csv('./clean_data/2020_election_by_county.csv')

vax = pd.read_csv('./clean_data/vaccine_hesitancy.csv')

In [75]:
vote.set_index('county_state', inplace=True)
vax.set_index('county_state', inplace=True)

In [76]:
vote[vote['state'] == 'ME']

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,,
franklin cty townships_me,ME,Joe Biden,DEM,31,21,52,0.404,0.596
indian township vtng dst_me,ME,Joe Biden,DEM,191,72,263,0.274,0.726
pleasant point votng dst_me,ME,Joe Biden,DEM,196,39,235,0.166,0.834
auburn_me,ME,Joe Biden,DEM,6482,5803,12285,0.472,0.528
durham_me,ME,Joe Biden,DEM,1361,1429,2790,0.512,0.488
...,...,...,...,...,...,...,...,...
north berwick_me,ME,Donald Trump,REP,1557,1554,3111,0.5,0.500
parsonsfield_me,ME,Donald Trump,REP,564,502,1066,0.529,0.471
shapleigh_me,ME,Donald Trump,REP,1034,755,1789,0.578,0.422


In [77]:
# Rename to match
## Jacqueline generated this dictionary

dict = {'aroostook cty townships_me':'aroostook_me',
'piscataquis cty townshps_me':'piscataquis_me',
'somerset cty townships_me':'somerset_me'}

vote.rename(index = dict, inplace=True)

### Alaska: has election districts that do not align with counties/boroughs

In [78]:
vote.shape

(4627, 8)

In [79]:
alaska_eds = vote[vote['state'] == 'AK']
alaska_eds.shape

(40, 8)

In [80]:
# Drop Alaska rows in vote df
vote.drop(list(alaska_eds.index), inplace=True)

In [81]:
vote.shape

(4587, 8)

In [82]:
boroughs = ['aleutians east borough_ak',
'aleutians west census area_ak',
'anchorage municipality_ak',
'bethel census area_ak',
'bristol bay borough_ak',
'denali borough_ak',
'dillingham census area_ak',
'fairbanks north star borough_ak',
'haines borough_ak',
'hoonah-angoon census area_ak',
'juneau city and borough_ak',
'kenai peninsula borough_ak',
'ketchikan gateway borough_ak',
'kodiak island borough_ak',
'kusilvak census area_ak',
'lake and peninsula borough_ak',
'matanuska-susitna borough_ak',
'nome census area_ak',
'north slope borough_ak',
'northwest arctic borough_ak',
'petersburg borough_ak',
'prince of wales-hyder census area_ak',
'sitka city and borough_ak',
'skagway municipality_ak',
'southeast fairbanks census area_ak',
'valdez-cordova_ak',
'wrangell city and borough_ak',
'yakutat city and borough_ak',
'yukon-koyukuk census area_ak',
#
            'chugach census area_ak', 'copper river census area_ak', 'valdez-cordova census area_ak', 'petersburg census area_ak']

In [83]:
# Define:

# Rows missing:
not_in_vote = [x for x in vax.index if x not in vote.index]

# Rows in vote df but not in other dfs
vote_excess_rows = [c for c in vote.index if c not in vax.index]

In [84]:
# Drop excess (mismatch) rows from vote df.

vote.drop(vote_excess_rows, inplace=True)

In [85]:
# Add AK county rows, fill with AK state data.

ak = []
for b in boroughs:
    ak.append({'county_state': b,
    'state': 'AK',
    'candidate_won': 'Donald Trump',
    'party': 'REP',
    'winner_votes': 189951,
    'nonwinner_votes':359530-189951, 
    'total_votes': 359530, 
    'pct_trump': round(189951/359530, 3), 
    'pct_biden': round(153778/359530, 3)
})
    
ak_rows = pd.DataFrame(ak)
ak_rows.set_index('county_state', inplace=True)

vote_names_fixed = pd.concat([vote, ak_rows])

In [86]:
vote_names_fixed.head()

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,,
kent_de,DE,Joe Biden,DEM,44552,42473,87025,0.488,0.512
new castle_de,DE,Joe Biden,DEM,195034,92599,287633,0.322,0.678
district of columbia_dc,DC,Joe Biden,DEM,39041,2640,41681,0.063,0.937
alachua_fl,FL,Joe Biden,DEM,89704,52900,142604,0.371,0.629
broward_fl,FL,Joe Biden,DEM,618752,338920,957672,0.354,0.646


### Fill in missing rows from New England states

In [87]:
[c for c in not_in_vote if c[::-1][:2][::-1] == 'me']

['androscoggin_me', 'kennebec_me', 'sagadahoc_me']

In [88]:
vote_names_fixed[vote_names_fixed['state'] == 'ME']

# Consult wikipedia and compare to df

# 13 of 16 Maine counties already in vote df
# 11 of 13 VT already in vote df
# 3 of 5 RI already in vote df
# 6 of 10 NH already in vote df
# 8 of 14 MA already in vote df
# 7 of 8 CT already in vote df

,state,candidate_won,party,winner_votes,nonwinner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,,
cumberland_me,ME,Joe Biden,DEM,4074,1928,6002,0.321,0.679
penobscot_me,ME,Joe Biden,DEM,512,333,845,0.394,0.606
york_me,ME,Joe Biden,DEM,6117,3541,9658,0.367,0.633
aroostook_me,ME,Donald Trump,REP,150,142,292,0.514,0.486
piscataquis_me,ME,Donald Trump,REP,61,26,87,0.701,0.299
somerset_me,ME,Donald Trump,REP,173,70,243,0.712,0.288
franklin_me,ME,Donald Trump,REP,459,402,861,0.533,0.467
hancock_me,ME,Donald Trump,REP,778,694,1472,0.529,0.471
washington_me,ME,Donald Trump,REP,540,448,988,0.547,0.453


In [89]:
# Fill in missing election data using wikipedia

new_england = [
# ME
    {'county_state': 'androscoggin_me',
    'state': 'ME',
    'candidate_won': 'Donald Trump',
    'party': 'REP',
    'winner_votes': 29268,
    'nonwinner_votes': 59361-29268, 
    'total_votes': 59361, 
    'pct_trump': .4985, 
    'pct_biden': .4704},
    {'county_state': 'kennebec_me',
     'state': 'ME',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 34721,
     'nonwinner_votes': 72615-34721,
     'total_votes':72615,
     'pct_trump': .4832,
     'pct_biden': .4857},
    {'county_state': 'sagadahoc_me',
     'state': 'ME',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 13528,
     'nonwinner_votes': 24280-13528,
     'total_votes':24280,
     'pct_trump': .4058,
     'pct_biden': .5628},
# VT
    {'county_state': 'caledonia_vt',
     'state': 'VT',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 9011,
     'nonwinner_votes': 16334-9011,
     'total_votes': 16334,
     'pct_trump': .401,
     'pct_biden': .552},
    {'county_state': 'lamoille_vt',
     'state': 'VT',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 10240,
     'nonwinner_votes': 15092-10240,
     'total_votes':15092,
     'pct_trump': .276,
     'pct_biden': .679},
    {'county_state': 'orleans_vt',
     'state': 'VT',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 7147,
     'nonwinner_votes': 14215-7147,
     'total_votes': 14215,
     'pct_trump': .458,
     'pct_biden': .503},
# RI
    {'county_state': 'kent_ri',
     'state': 'RI',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 18050,
     'nonwinner_votes': 93093-18050,
     'total_votes': 93093,
     'pct_trump': .4512,
     'pct_biden': .5276},
    {'county_state': 'washington_ri',
     'state': 'RI',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 44549,
     'nonwinner_votes': 76060-44549,
     'total_votes': 76060,
     'pct_trump': .3920,
     'pct_biden': .5857},
# NH
   {'county_state': 'belknap_nh',
     'state': 'NH',
     'candidate_won': 'Donald Trump',
     'party': 'REP',
     'winner_votes': 20899,
     'nonwinner_votes': 38479-20899,
     'total_votes': 38479,
     'pct_trump': .5431,
     'pct_biden': .439},
    {'county_state': 'cheshire_nh',
     'state': 'NH',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 25522,
     'nonwinner_votes': 44370-25522,
     'total_votes': 44370,
     'pct_trump': .4034,
     'pct_biden': .5752},
    {'county_state': 'coos_nh',
     'state': 'NH',
     'candidate_won': 'Donald Trump',
     'party': 'REP',
     'winner_votes': 8617,
     'nonwinner_votes': 16544-8617,
     'total_votes': 16544,
     'pct_trump': .5209,
     'pct_biden': .4618},
    {'county_state': 'rockingham_nh',
     'state': 'NH',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 100064,
     'nonwinner_votes': 199342-100064,
     'total_votes': 199342,
     'pct_trump': .4809,
     'pct_biden': .5020},
# MA
    {'county_state': 'berkshire_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 51705,
     'nonwinner_votes': 71771-51705,
     'total_votes': 71771,
     'pct_trump': .2517,
     'pct_biden': .7204},
    {'county_state': 'bristol_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 153377,
     'nonwinner_votes': 281364-153377,
     'total_votes': 281364,
     'pct_trump': .426,
     'pct_biden': .5451},
    {'county_state': 'dukes_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 9914,
     'nonwinner_votes': 12873-9914,
     'total_votes': 12873,
     'pct_trump': .2044,
     'pct_biden': .7704},
    {'county_state': 'hampshire_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 63362,
     'nonwinner_votes': 88332-63362,
     'total_votes': 88332,
     'pct_trump': .2522,
     'pct_biden': .7173},
    {'county_state': 'middlesex_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 617196,
     'nonwinner_votes': 869347-617196,
     'total_votes': 869347,
     'pct_trump': .2611,
     'pct_biden': .71},
    {'county_state': 'suffolk_ma',
     'state': 'MA',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 270522,
     'nonwinner_votes': 337240-270522,
     'total_votes': 337240,
     'pct_trump': .1738,
     'pct_biden': .8022},
#CT
    {'county_state': 'middlesex_ct',
     'state': 'CT',
     'candidate_won': 'Joe Biden',
     'party': 'DEM',
     'winner_votes': 56848,
     'nonwinner_votes': 99203-55848,
     'total_votes': 99203,
     'pct_trump': .4099,
     'pct_biden': .573}
]

In [90]:
ne_df = pd.DataFrame(new_england)
ne_df.set_index('county_state', inplace=True)

In [91]:
big_ready = pd.concat([vote_names_fixed, ne_df])

In [92]:
# Drop columns we won't be using (rue the inefficiency of not doing this earlier)

big_ready.drop(columns='nonwinner_votes', inplace=True)

In [93]:
big_ready.shape

(3145, 7)

In [94]:
big_ready.head()

,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,
kent_de,DE,Joe Biden,DEM,44552,87025,0.488,0.512
new castle_de,DE,Joe Biden,DEM,195034,287633,0.322,0.678
district of columbia_dc,DC,Joe Biden,DEM,39041,41681,0.063,0.937
alachua_fl,FL,Joe Biden,DEM,89704,142604,0.371,0.629
broward_fl,FL,Joe Biden,DEM,618752,957672,0.354,0.646


In [95]:
big_ready.to_csv('./clean_data/final_clean_vote_data.csv')

In [96]:
big_ready.head()

,state,candidate_won,party,winner_votes,total_votes,pct_trump,pct_biden
county_state,,,,,,,
kent_de,DE,Joe Biden,DEM,44552,87025,0.488,0.512
new castle_de,DE,Joe Biden,DEM,195034,287633,0.322,0.678
district of columbia_dc,DC,Joe Biden,DEM,39041,41681,0.063,0.937
alachua_fl,FL,Joe Biden,DEM,89704,142604,0.371,0.629
broward_fl,FL,Joe Biden,DEM,618752,957672,0.354,0.646
